In [1]:
import torch
import torchvision
import torchvision.transforms as T
import pandas as pd
from PIL import Image
import os
import logging
from torch.utils.data import Dataset
import torch.nn as nn

print("PyTorch version:", torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

PyTorch version: 2.5.1+cu118
Device: cuda


In [2]:
acuity_path = "../Choithram Netralaya Data/acuityvalues.xlsx"

df = pd.read_excel(acuity_path)

display(df.head())

print("Columns in the dataset:")
print(df.columns)

,patient,r sphere,r cylinder,l sphere,l cylinder
0,1,0.25,-0.25,0.00,-0.25
1,2,-0.50,-0.50,0.00,-0.50
2,3,1.75,-0.75,0.00,-0.25
3,4,0.25,0.00,0.25,0.00
4,5,0.25,0.25,0.25,0.00


Columns in the dataset:
Index(['patient', 'r sphere', 'r cylinder', 'l sphere', 'l cylinder'], dtype='object')


In [3]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

filtered_df = df.copy()
numeric_cols = ['r sphere', 'r cylinder', 'l sphere', 'l cylinder']

filtered_df = filtered_df.dropna(subset=numeric_cols)

df_patients = set(map(str, filtered_df['patient'].unique()))

data_dir = "../Choithram Netralaya Data/Images"
image_patients = set()
if os.path.exists(data_dir):
    for folder_name in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder_name)
        if os.path.isdir(folder_path):
            image_patients.add(folder_name)
else:
    logging.warning("Image directory does not exist!")

all_patients = df_patients.union(image_patients)

valid_patients = []
logging.info(f"Total patients in DF (after NaN drop): {len(df_patients)}")
logging.info(f"Total patients in image directory: {len(image_patients)}")
logging.info(f"Combined unique patients: {len(all_patients)}")

for patient_id in sorted(all_patients, key=lambda x: int(x)):
    if patient_id not in df_patients:
        logging.info(f"Skipping patient {patient_id}: no numeric values found in DataFrame.")
        continue
    row = filtered_df.loc[filtered_df['patient'] == int(patient_id)].head(1)
    if row.empty:
        logging.info(f"Skipping patient {patient_id}: filtered out due to missing numeric data.")
        continue

    left_ir_path = os.path.join(data_dir, patient_id, f"{patient_id}_LEFT_IR.jpg")
    right_ir_path = os.path.join(data_dir, patient_id, f"{patient_id}_RIGHT_IR.jpg")

    left_exists = os.path.exists(left_ir_path)
    right_exists = os.path.exists(right_ir_path)

    if not left_exists and not right_exists:
        logging.info(f"Skipping patient {patient_id}: LEFT_IR and RIGHT_IR images missing.")
        continue
    elif not left_exists:
        logging.info(f"Skipping patient {patient_id}: LEFT_IR image missing.")
        continue
    elif not right_exists:
        logging.info(f"Skipping patient {patient_id}: RIGHT_IR image missing.")
        continue

    logging.info(f"Including patient {patient_id}: Numeric and IR images available.")
    valid_patients.append(patient_id)

valid_patient_ids = set(map(int, valid_patients))
filtered_df = filtered_df[filtered_df['patient'].isin(valid_patient_ids)].reset_index(drop=True)

logging.info(f"Number of valid patients after filtering: {len(filtered_df)}")

INFO: Total patients in DF (after NaN drop): 302
INFO: Total patients in image directory: 343
INFO: Combined unique patients: 343
INFO: Skipping patient 1: LEFT_IR image missing.
INFO: Including patient 2: Numeric and IR images available.
INFO: Including patient 3: Numeric and IR images available.
INFO: Including patient 4: Numeric and IR images available.
INFO: Including patient 5: Numeric and IR images available.
INFO: Including patient 6: Numeric and IR images available.
INFO: Including patient 7: Numeric and IR images available.
INFO: Including patient 8: Numeric and IR images available.
INFO: Including patient 9: Numeric and IR images available.
INFO: Including patient 10: Numeric and IR images available.
INFO: Including patient 11: Numeric and IR images available.
INFO: Including patient 12: Numeric and IR images available.
INFO: Including patient 13: Numeric and IR images available.
INFO: Including patient 14: Numeric and IR images available.
INFO: Including patient 15: Numeric 

In [5]:
class EyeIRDataset(Dataset):
    def __init__(self, df, data_dir, transform=None, is_training=True):
        """
        Args:
            df (DataFrame): Data containing patient IDs and numerical features.
            data_dir (str): Directory where images are stored.
            transform: Image transformations.
            is_training (bool): Whether the dataset is for training or testing.
        """
        self.df = df
        self.data_dir = data_dir
        self.transform = transform
        self.is_training = is_training

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Use patient_id internally for loading images
        patient_id = str(int(row['patient']))
        left_ir_path = os.path.join(self.data_dir, patient_id, f"{patient_id}_LEFT_IR.jpg")
        right_ir_path = os.path.join(self.data_dir, patient_id, f"{patient_id}_RIGHT_IR.jpg")

        try:
            left_img = Image.open(left_ir_path).convert("RGB")
            right_img = Image.open(right_ir_path).convert("RGB")
        except (FileNotFoundError, IOError) as e:
            print(f"Error loading image for patient {patient_id}: {e}")
            left_img = Image.new("RGB", (240, 240), (0, 0, 0))
            right_img = Image.new("RGB", (240, 240), (0, 0, 0))

        if self.transform:
            left_img = self.transform(left_img)
            right_img = self.transform(right_img)

        if self.is_training:
            numeric_features = torch.tensor(
                [row['r sphere'], row['r cylinder'], row['l sphere'], row['l cylinder']],
                dtype=torch.float
            )
            target = numeric_features.clone()

            # Add weights
            weights = torch.tensor(
                [row['r_weight'], row['c_weight'], row['l_weight'], row['lc_weight']],
                dtype=torch.float
            )

            return {
                "left_ir": left_img,
                "right_ir": right_img,
                "numeric_features": numeric_features,
                "target": target,
                "weights": weights
            }
        else:
            return {
                "left_ir": left_img,
                "right_ir": right_img
            }


NameError: name 'train_dataset' is not defined

In [5]:
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()
])

data_dir = "../Choithram Netralaya Data/Images"
dataset = EyeIRDataset(filtered_df, data_dir, transform=transform)

In [6]:
# Check a single sample from the dataset
sample = dataset[0]
print("Left IR Shape:", sample['left_ir'].shape)
print("Right IR Shape:", sample['right_ir'].shape)
print("Numeric Features:", sample['numeric_features'])
print("Target:", sample['target'])


Left IR Shape: torch.Size([3, 224, 224])
Right IR Shape: torch.Size([3, 224, 224])
Numeric Features: tensor([-0.5000, -0.5000,  0.0000, -0.5000])
Target: tensor([-0.5000, -0.5000,  0.0000, -0.5000])


In [7]:
from torch.utils.data import DataLoader

# Split dataset into training and testing
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create DataLoaders
batch_size = 16  # Adjust based on your hardware capabilities
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Verify a batch from DataLoader
for batch in train_loader:
    print("Left IR Batch Shape:", batch['left_ir'].shape)
    print("Right IR Batch Shape:", batch['right_ir'].shape)
    print("Numeric Features Batch Shape:", batch['numeric_features'].shape)
    print("Target Batch Shape:", batch['target'].shape)
    break

Left IR Batch Shape: torch.Size([16, 3, 224, 224])
Right IR Batch Shape: torch.Size([16, 3, 224, 224])
Numeric Features Batch Shape: torch.Size([16, 4])
Target Batch Shape: torch.Size([16, 4])


In [13]:
import torch
import torch.nn as nn
from torchvision.models import efficientnet_b1, EfficientNet_B1_Weights

# Define the Eye Model with EfficientNet B1
class EyeModel(nn.Module):
    def __init__(self, output_dim=4):
        super(EyeModel, self).__init__()
        # Load EfficientNet B1
        self.cnn = efficientnet_b1(weights=EfficientNet_B1_Weights.IMAGENET1K_V1)
        self.cnn.classifier = nn.Identity()  # Remove classifier to extract features

        # Fully connected layers for image features
        self.img_compress = nn.Sequential(
            nn.Linear(1280 * 2, 512),  # 1280 features per image x 2 (left and right IR)
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        # MLP for numeric features
        self.numeric_branch = nn.Sequential(
            nn.Linear(4, 64),  # 4 numeric inputs
            nn.ReLU(),
            nn.Linear(64, 32)
        )

        # Fully connected layers for final prediction
        self.fc = nn.Sequential(
            nn.Linear(512 + 32, 128),  # Combine image and numeric features
            nn.ReLU(),
            nn.Dropout(0.6),
            nn.Linear(128, output_dim)  # Output: [r sphere, r cylinder, l sphere, l cylinder]
        )

    def forward(self, left_ir, right_ir, numeric):
        # Image feature extraction
        left_feat = self.cnn(left_ir)
        right_feat = self.cnn(right_ir)

        # Combine and compress image features
        combined_img_feat = torch.cat([left_feat, right_feat], dim=1)
        combined_img_feat = self.img_compress(combined_img_feat)

        # Process numeric features
        numeric_feat = self.numeric_branch(numeric)

        # Combine all features for final prediction
        fused_feat = torch.cat([combined_img_feat, numeric_feat], dim=1)
        out = self.fc(fused_feat)
        return out

# Initialize the model
model = EyeModel().to(device)
print(model)


EyeModel(
  (cnn): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scale_activati

In [14]:
import torch.optim as optim

# Define the loss function
# Define the weights for each feature
weights = torch.tensor([0.5, 0.3, 0.1, 0.1]).to(device)  # Adjust these weights as necessary

# Custom weighted loss function
def weighted_mse_loss(predictions, targets, weights):
    mse = (predictions - targets) ** 2
    weighted_mse = weights * mse  # Apply weights to each feature
    return weighted_mse.mean()

# Use the custom loss function in the training loop
criterion = lambda preds, targets: weighted_mse_loss(preds, targets, weights)

# Define the optimizer
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

# Define the learning rate scheduler
# scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)  # Adjust T_max based on epochs


scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)
# Print optimizer and scheduler details for verification
print(optimizer)
print(scheduler)

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 0.0001
    lr: 0.0001
    maximize: False
    weight_decay: 1e-05
)


In [4]:
import numpy as np

# Metric calculation function (overall)
def calculate_metrics(predictions, targets, weights=None):
    """
    Calculates overall metrics: Mean Absolute Error (MAE), Root Mean Squared Error (RMSE), and R-squared (R²).
    Args:
        predictions (Tensor): Predicted values from the model.
        targets (Tensor): Ground truth values.
        weights (Tensor, optional): Weights for weighted metrics.
    Returns:
        tuple: MAE, RMSE, R² for the entire batch.
    """
    predictions = predictions.detach().cpu().numpy()
    targets = targets.detach().cpu().numpy()
    weights = weights.detach().cpu().numpy() if weights is not None else np.ones_like(targets)

    mae = np.mean(weights * np.abs(predictions - targets))
    rmse = np.sqrt(np.mean(weights * (predictions - targets) ** 2))
    total_variance = np.sum(weights * (targets - np.mean(targets)) ** 2)
    explained_variance = np.sum(weights * (predictions - targets) ** 2)
    r_squared = 1 - (explained_variance / total_variance) if total_variance != 0 else 0

    return mae, rmse, r_squared


# Metric calculation function (feature-specific)
def calculate_feature_metrics(predictions, targets):
    """
    Calculates metrics per output feature: MAE, RMSE, and R².
    """
    predictions = predictions.detach().cpu().numpy()
    targets = targets.detach().cpu().numpy()
    feature_metrics = {}

    for i, feature in enumerate(['r sphere', 'r cylinder', 'l sphere', 'l cylinder']):
        pred = predictions[:, i]
        tgt = targets[:, i]
        mae = np.mean(np.abs(pred - tgt))
        rmse = np.sqrt(np.mean((pred - tgt) ** 2))
        r_squared = 1 - (np.sum((tgt - pred) ** 2) / np.sum((tgt - np.mean(tgt)) ** 2))
        feature_metrics[feature] = {'mae': mae, 'rmse': rmse, 'r2': r_squared}

    return feature_metrics


# Early stopping class
class EarlyStopping:
    def __init__(self, patience=5, delta=0.01, verbose=True):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
        self.verbose = verbose

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                if self.verbose:
                    print("Early stopping triggered!")
        else:
            self.best_loss = val_loss
            self.counter = 0


# Training loop
epochs = 30
early_stopping = EarlyStopping(patience=5, delta=0.01)
best_val_loss = float("inf")

for epoch in range(epochs):
    # Training phase
    model.train()
    train_loss, train_mae, train_rmse, train_r2 = 0, 0, 0, 0
    train_feature_metrics = {'r sphere': {'mae': 0, 'rmse': 0, 'r2': 0},
                             'r cylinder': {'mae': 0, 'rmse': 0, 'r2': 0},
                             'l sphere': {'mae': 0, 'rmse': 0, 'r2': 0},
                             'l cylinder': {'mae': 0, 'rmse': 0, 'r2': 0}}

    for batch in train_loader:
        left_ir = batch['left_ir'].to(device)
        right_ir = batch['right_ir'].to(device)
        numeric = batch['numeric_features'].to(device)
        target = batch['target'].to(device)
        weights = batch['weights'].to(device)

        optimizer.zero_grad()
        predictions = model(left_ir, right_ir, numeric)
        loss = criterion(predictions, target, weights)  # Weighted loss function
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        mae, rmse, r2 = calculate_metrics(predictions, target, weights)
        train_mae += mae
        train_rmse += rmse
        train_r2 += r2

        # Feature-specific metrics
        batch_feature_metrics = calculate_feature_metrics(predictions, target)
        for feature in train_feature_metrics:
            train_feature_metrics[feature]['mae'] += batch_feature_metrics[feature]['mae']
            train_feature_metrics[feature]['rmse'] += batch_feature_metrics[feature]['rmse']
            train_feature_metrics[feature]['r2'] += batch_feature_metrics[feature]['r2']

    train_loss /= len(train_loader)
    train_mae /= len(train_loader)
    train_rmse /= len(train_loader)
    train_r2 /= len(train_loader)
    for feature in train_feature_metrics:
        train_feature_metrics[feature]['mae'] /= len(train_loader)
        train_feature_metrics[feature]['rmse'] /= len(train_loader)
        train_feature_metrics[feature]['r2'] /= len(train_loader)

    # Validation phase
    model.eval()
    val_loss, val_mae, val_rmse, val_r2 = 0, 0, 0, 0
    val_feature_metrics = {'r sphere': {'mae': 0, 'rmse': 0, 'r2': 0},
                           'r cylinder': {'mae': 0, 'rmse': 0, 'r2': 0},
                           'l sphere': {'mae': 0, 'rmse': 0, 'r2': 0},
                           'l cylinder': {'mae': 0, 'rmse': 0, 'r2': 0}}

    with torch.no_grad():
        for batch in test_loader:
            left_ir = batch['left_ir'].to(device)
            right_ir = batch['right_ir'].to(device)
            numeric = batch['numeric_features'].to(device)
            target = batch['target'].to(device)
            weights = batch['weights'].to(device)

            predictions = model(left_ir, right_ir, numeric)
            loss = criterion(predictions, target, weights)

            val_loss += loss.item()
            mae, rmse, r2 = calculate_metrics(predictions, target, weights)
            val_mae += mae
            val_rmse += rmse
            val_r2 += r2

            # Feature-specific metrics
            batch_feature_metrics = calculate_feature_metrics(predictions, target)
            for feature in val_feature_metrics:
                val_feature_metrics[feature]['mae'] += batch_feature_metrics[feature]['mae']
                val_feature_metrics[feature]['rmse'] += batch_feature_metrics[feature]['rmse']
                val_feature_metrics[feature]['r2'] += batch_feature_metrics[feature]['r2']

    val_loss /= len(test_loader)
    val_mae /= len(test_loader)
    val_rmse /= len(test_loader)
    val_r2 /= len(test_loader)
    for feature in val_feature_metrics:
        val_feature_metrics[feature]['mae'] /= len(test_loader)
        val_feature_metrics[feature]['rmse'] /= len(test_loader)
        val_feature_metrics[feature]['r2'] /= len(test_loader)

    # Adjust learning rate
    scheduler.step(val_loss)

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        # torch.save(model.state_dict(), f"best_model_epoch_{epoch+1}.pth")

    # Print metrics for this epoch
    print(
        f"Epoch [{epoch+1}/{epochs}], LR: {scheduler.get_last_lr()[0]:.6f}\n"
        f"Train Loss: {train_loss:.4f}, MAE: {train_mae:.4f}, RMSE: {train_rmse:.4f}, R²: {train_r2:.4f}\n"
        f"Val Loss: {val_loss:.4f}, MAE: {val_mae:.4f}, RMSE: {val_rmse:.4f}, R²: {val_r2:.4f}\n"
    )
    print("Feature-Specific Metrics (Train):")
    for feature, metrics in train_feature_metrics.items():
        print(f"  {feature}: MAE: {metrics['mae']:.4f}, RMSE: {metrics['rmse']:.4f}, R²: {metrics['r2']:.4f}")
    print("Feature-Specific Metrics (Validation):")
    for feature, metrics in val_feature_metrics.items():
        print(f"  {feature}: MAE: {metrics['mae']:.4f}, RMSE: {metrics['rmse']:.4f}, R²: {metrics['r2']:.4f}")

    # Early stopping
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered!")
        break

NameError: name 'model' is not defined